<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTING LIBRARIES:**

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import math
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#https://drive.google.com/file/d/1HcaON_5Qz-bfqlRyDgVWfdS4bD0hQ0Z3/view?usp=sharing

In [3]:
id='1HcaON_5Qz-bfqlRyDgVWfdS4bD0hQ0Z3'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('MoviesData.csv')  
movies_data = pd.read_csv('MoviesData.csv',engine='python',index_col=0)
movies_data.head()

,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,profit,production_countries,status,popularity,vote_average,vote_count,movieId,cast,director,rating_count,mean_rating
0,Se7en,1995,Friday,Crime|Mystery|Thriller,English,127.0,Two homicide detectives are on a desperate hun...,New Line Cinema|Juno Pix|Cecchi Gori Pictures,33000000,327311859.0,294311859.0,United States of America,Released,18.457430,8.1,5915.0,47,Brad Pitt|Morgan Freeman|Gwyneth Paltrow|John ...,David Fincher,91921.0,4.052926
1,Batman,1989,Friday,Fantasy|Action,English,126.0,The Dark Knight of Gotham City begins his war ...,PolyGram Filmed Entertainment|The Guber-Peters...,35000000,411348924.0,376348924.0,United Kingdom|United States of America,Released,19.106730,7.0,2145.0,592,Jack Nicholson|Michael Keaton|Kim Basinger|Mic...,Tim Burton,91082.0,4.429015
2,The Empire Strikes Back,1980,Saturday,Adventure|Action|Science Fiction,English,124.0,"The epic saga continues as Luke Skywalker, in ...",Lucasfilm|Twentieth Century Fox Film Corporation,18000000,538400000.0,520400000.0,United States of America,Released,19.470959,8.2,5998.0,1196,Mark Hamill|Harrison Ford|Carrie Fisher|Billy ...,Irvin Kershner,87901.0,4.169975
3,Drag Me to Hell,2009,Sunday,Horror|Thriller,English,99.0,After denying a woman the extension she needs ...,Universal Pictures|Ghost House Pictures|Mandat...,30000000,90810892.0,60810892.0,United States of America,Released,15.809588,6.2,995.0,68952,Alison Lohman|Justin Long|Lorna Raver|Dileep R...,Sam Raimi,84078.0,4.152246
4,Suspect,1987,Friday,Crime|Drama|Thriller,English,121.0,When a Supreme Court judge commits suicide and...,TriStar Pictures,8000000,18782400.0,10782400.0,United States of America,Released,10.638289,6.4,45.0,4332,Cher|Dennis Quaid|Liam Neeson|John Mahoney|Joe...,Peter Yates,77960.0,4.154098


# **POPULARITY BASED RECOMMENDATION SYSTEM :**

In [13]:
movies_data[['vote_count','vote_average','rating_count','mean_rating','popularity']].describe()

,vote_count,vote_average,rating_count,mean_rating,popularity
count,45663.000000,45663.000000,45663.000000,45663.000000,45663.000000
mean,109.588179,5.617947,484.200666,0.771628,2.922354
std,490.228641,1.924399,2943.748802,1.389116,5.996711
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,5.000000,0.000000,0.000000,0.386454
50%,10.000000,6.000000,0.000000,0.000000,1.129126
75%,34.000000,6.800000,0.000000,0.000000,3.685682
max,14075.000000,10.000000,91921.000000,5.000000,547.488298


**WEIGHTED AVERAGE FOR EACH MOVIES AVERAGE RATINGS :**

Weighted Ratings: 
>              W =   Rv + Cm
>                   ---------
                     v+m

R= average rating for the movie as a number from 0 to 10

v= number of votes for the movie

m= minimum votes required to be listed in Top(here 2000)

C=the mean vote across whole report


In [11]:
R=movies_data['vote_average']
v=movies_data['vote_count']
C=movies_data['vote_average'].mean()
m=2000

In [12]:
movies_data['weighted_ratings']=(R*v+C*m)/(v+m)
movies_data[['title','weighted_ratings']].sort_values('weighted_ratings',ascending=False).head(50)

,title,weighted_ratings
67,The Shawshank Redemption,7.943512
1555,The Dark Knight,7.924073
2120,Fight Club,7.840666
2130,Pulp Fiction,7.797272
6075,Inception,7.791191
9040,The Godfather,7.781642
33564,Interstellar,7.723561
109,Forrest Gump,7.691071
2594,The Lord of the Rings: The Return of the King,7.614560
1942,The Lord of the Rings: The Fellowship of the Ring,7.562605
